In [ ]:
execfile('D:/measuring/analysis/scripts/setup_analysis.py')
%matplotlib inline
import os
import numpy as np
import scipy
import sys
import matplotlib
import matplotlib.pyplot as plt
import math
import scipy.constants
c=scipy.constants.c #speed of light
import time
filepath='D:/measuring/'
import scipy.signal
os.chdir(filepath)

sys.path.append('H:/My Documents/measuring/')

#data_dir='D:\measuring\data/20160801/spectometer dataspectrometer5'#:\measuring\data/20160503\ON diamond\pos 7/raw data spectrometer\L9'#'D:\measuring\data/20160707\CL2_newmirror_processed'#"/Users/suzannevandam/Documents/PhD/localdata/20160430/ON_diamond" 
#data_dir=r'D:\measuring\data/20170213/spectrometer/180600_cav_length_11'#:\measuring\data/20160503\ON diamond\pos 7/raw data spectrometer\L9'#'D:\measuring\data/20160707\CL2_newmirror_processed'#"/Users/suzannevandam/Documents/PhD/localdata/20160430/ON_diamond" 
data_dir=r'D:\measuring\data\20170816\190000_Spot_in_RID_irr' #181600_export3'#2017-05-17 10_31_18 Hans_SIL4 4.csv'
bg_data_dir=r'D:\measuring\data\20170803/154000'
subtract_bg=False
nr_bg_subtraction = 1.0 #time measurement / time_bg
print data_dir

%matplotlib inline

In [ ]:
import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as spectrometer_analysis_2Dsweep

In [ ]:
def extract_freq_data(data_dir):
    sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,load_analysis_from_file=False)#,V_min=ana_pars['V_min'],V_max=ana_pars['V_max'],laser_wavelength=ana_pars['laser_wavelength'])
    sa_2D.ana_pars['min_frq']=100
    frequencies,filenumbers,intensities=sa_2D.get_data()
    return frequencies, intensities

In [ ]:
reload(spectrometer_analysis_2Dsweep)
reload(sa)

#a = sa.spectrometer_analysis(data_dir)
#frequencies,fn,intensity =a.load_data_from_folder()
bg_sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(bg_data_dir,load_analysis_from_file=False)#,V_min=ana_pars['V_min'],V_max=ana_pars['V_max'],laser_wavelength=ana_pars['laser_wavelength'])
bg_sa_2D.ana_pars['min_frq']=100
bg_frequencies,bg_filenumbers,bg_intensities=bg_sa_2D.get_data()
#frequencies,filenumbers,intensities = sa_2D.get_data()#min_frq=ana_pars['min_frq'],max_frq=ana_pars['max_frq'])#load_data_from_folder(data_dir)
#intensities = sa_2D.subtract_offset()
#print frequencies, intensity
# sa_2D.save_analysis_params(ana_pars)
bg_sa_2D.save_analysis()


bg_cts_gg_637 = np.sum(bg_intensities[np.where(bg_frequencies<(c/638e-9*1.e-12))])
bg_cts_ll_637 = np.sum(bg_intensities[np.where(bg_frequencies>(c/638e-9*1.e-12))])
bg_cts_636_638 = np.sum(bg_intensities[np.where(((bg_frequencies>(c/638e-9*1.e-12))&(bg_frequencies<(c/636e-9*1.e-12))))])
bg_total_cts = np.sum(bg_intensities)
bg_total_cts_2 = bg_cts_gg_637+bg_cts_ll_637
bg_text= 'cts > 637 nm: '+str(int(bg_cts_gg_637))+' ('+str(round(bg_cts_gg_637/bg_total_cts*100,1))+'%) \ncts < 637 nm: '+str(int(cts_ll_637))+' ('+str(round(cts_ll_637/total_cts*100,1))+'%) \n636 nm < cts < 638 nm: '+str(int(cts_636_638))+' ('+str(round(cts_636_638/total_cts*100,1))+'%)'
print bg_text
# print 'total counts above 637 nm: ',int(cts_gg_637),' (',round(cts_gg_637/total_cts*100,1),'%)'
# print 'total counts below 637 nm: ',int(cts_ll_637),' (',round(cts_ll_637/total_cts*100,1),'%)'
# print 'total counts between 636 and 638 nm: ',int(cts_636_638),' (',round(cts_636_638/total_cts*100,1),'%)'

    

In [ ]:
reload(spectrometer_analysis_2Dsweep)
reload(sa)

#a = sa.spectrometer_analysis(data_dir)
#frequencies,fn,intensity =a.load_data_from_folder()
sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,load_analysis_from_file=False)#,V_min=ana_pars['V_min'],V_max=ana_pars['V_max'],laser_wavelength=ana_pars['laser_wavelength'])
sa_2D.ana_pars['min_frq']=100
frequencies,filenumbers,intensities=sa_2D.get_data()
#frequencies,filenumbers,intensities = sa_2D.get_data()#min_frq=ana_pars['min_frq'],max_frq=ana_pars['max_frq'])#load_data_from_folder(data_dir)
#intensities = sa_2D.subtract_offset()
#print frequencies, intensity
# sa_2D.save_analysis_params(ana_pars)
sa_2D.save_analysis()


cts_gg_637 = np.sum(intensities[np.where(frequencies<(c/638e-9*1.e-12))])
cts_ll_637 = np.sum(intensities[np.where(frequencies>(c/638e-9*1.e-12))])
cts_636_638 = np.sum(intensities[np.where(((frequencies>(c/638e-9*1.e-12))&(frequencies<(c/636e-9*1.e-12))))])
total_cts = np.sum(intensities)
total_cts_2 = cts_gg_637+cts_ll_637
text= 'cts > 637 nm: '+str(int(cts_gg_637))+' ('+str(round(cts_gg_637/total_cts*100,1))+'%) \ncts < 637 nm: '+str(int(cts_ll_637))+' ('+str(round(cts_ll_637/total_cts*100,1))+'%) \n636 nm < cts < 638 nm: '+str(int(cts_636_638))+' ('+str(round(cts_636_638/total_cts*100,1))+'%)'
print text
# print 'total counts above 637 nm: ',int(cts_gg_637),' (',round(cts_gg_637/total_cts*100,1),'%)'
# print 'total counts below 637 nm: ',int(cts_ll_637),' (',round(cts_ll_637/total_cts*100,1),'%)'
# print 'total counts between 636 and 638 nm: ',int(cts_636_638),' (',round(cts_636_638/total_cts*100,1),'%)'


In [ ]:
frequencies,intensities = extract_freq_data(data_dir)
intensities = np.average(intensities,axis=1)
bg_frequencies,bg_intensities = extract_freq_data(bg_data_dir)
bg_intensities = np.average(bg_intensities,axis=1)
nr_bg_subtraction = 1.0

if subtract_bg:
    intensities = (intensities-np.average(intensities))-nr_bg_subtraction*(bg_intensities-np.average(bg_intensities))
    print 'Background file:', bg_data_dir

plot_freq_domain(frequencies,intensities[:],(430,480),False)
plot_wavelength_domain(frequencies,intensities[:],(620,800))

print np.shape(intensities)



### Plotting in the frequency domain
With this function the data can be plotted in the frequency domain. Additionally the data can be fitted with a Lorentzian curve with one or two peaks.

In [ ]:
def plot_freq_domain(freqencies,intensities,(fmin,fmax),fit_data=False):
    fig,ax=plt.subplots()
    freq_in_GHz = frequencies[::-1]#(frequencies[::-1]-470.4)*1e3
    intensities = intensities[::-1]
    
    


    if fit_data:
        g_a= 2.0
        g_A= 12.0
        g_x0 = -275.0# -180
        g_gamma = 25
        g_A2= 10.0
        g_x02 =-210.0# -380
        g_gamma2 = 25
        fixed=[]
    
        idx_min = np.searchsorted(freq_in_GHz, fmin, side="left")
        idx_max = np.searchsorted(freq_in_GHz, fmax, side="left")
        freq_in_GHz = freq_in_GHz[idx_min:idx_max]
        intensities = intensities[idx_min:idx_max]

        
        p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
#         p0, fitfunc, fitfunc_str = common.fit_2lorentz(g_a, g_A, g_x0, g_gamma,g_A2, g_x02, g_gamma2)
        fit_result = fit.fit1d(freq_in_GHz,intensities, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)
#         fit_result = fit.fit1d(freq_in_GHz,(intensities[:]-np.average(intensities)), None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)

#         T1 = fit_result['params_dict']['x01']
#         gamma1 = fit_result['params_dict']['gamma1']
#         u_gamma1 = fit_result['error_dict']['gamma1']
#         print 'frequency = ', T1 , 'GHz'
#         print 'linewidth= ', gamma1,'+/-', u_gamma1, 'GHz'

#         T2 = fit_result['params_dict']['x02']
#         gamma2 = fit_result['params_dict']['gamma2']
#         u_gamma2 = fit_result['error_dict']['gamma2']

#         print 'frequency = ', T2 , 'GHz'
#         print 'linewidth= ', gamma2,'+/-', u_gamma2, 'GHz'

#         print 'T1-T2=' ,(T1-T2)


        plot.plot_fit1d(fit_result, np.linspace(min(freq_in_GHz),max(freq_in_GHz),len(freq_in_GHz)),ax=ax, label='Fit',show_guess=False, plot_data=True,color='red', data_linestyle = '-', print_info= False)


    ax.plot(freq_in_GHz,intensities[:],label='data')
    ax.set_xlabel('frequency (GHz wrt 470.4) ')
    ax.set_ylabel('avg counts per camera row in ROI')
    ax.set_xlim((fmin,fmax))
    ymin = np.amin(intensities[np.where((freq_in_GHz>fmin)&(freq_in_GHz<fmax))])-1
    ymax = np.amax(intensities[np.where((freq_in_GHz>fmin)&(freq_in_GHz<fmax))])+1
    ax.set_ylim([600,2400])#([ymin,ymax])
    ax.set_title(data_dir)
    #ax.text(500,1000,text)
    # plt.legend(loc='upper left')
    plt.savefig(os.path.join(data_dir,'plot.png'))


### Plotting the data in the wavelength domain

In [ ]:
def plot_wavelength_domain(frequencies,intensities,(xmin,xmax)):
    fig,ax=plt.subplots()
    wavelengths=c/frequencies*1.e-3
    ax.plot(wavelengths,(intensities[:]),label='data')
    ax.set_xlabel('wavelength (nm)')
    ax.set_ylabel('avg counts per camera row in ROI')
    #ax.set_ylim([640,680])
    ax.set_xlim((xmin,xmax))
    ymin = np.amin(intensities[np.where((wavelengths>xmin) & (wavelengths<xmax))])-1
    ymax = np.amax(intensities[np.where((wavelengths>xmin) & (wavelengths<xmax))])+1
    ax.set_ylim([ymin,ymax])
    ax.set_title(data_dir)
    #ax.text(500,1000,text)
    # plt.legend(loc='upper left')
    plt.savefig(os.path.join(data_dir,'plot.png'))


## Extract cavity length from cavity spectrometer data

In [ ]:
reload(spectrometer_analysis_2Dsweep)
reload(sa)

data_dir=r'D:\measuring\data\20170928\142600_cav_length'

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,load_analysis_from_file=False)#,V_min=ana_pars['V_min'],V_max=ana_pars['V_max'],laser_wavelength=ana_pars['laser_wavelength'])
sa_2D.ana_pars['min_frq']=440
sa_2D.ana_pars['max_frq']=500

frequencies,filenumbers,intensities=sa_2D.get_data()

plt.plot(c/(frequencies*1.e12)*1e9,intensities)


cavlength_minrange = 1
cavlength_maxrange = 100 # 20
minimum_peak_height = 2 # 100
minimum_peak_distance = 0.1 # 1
hom_max = 0.1 # 5.0 # 7.0


max_gamma = 10
x0s,u_x0s,n_trans, u_n_trans =sa_2D.peaks_from_1D_data(intensity=sa_2D.intensities[:,-1],plot_peak_locations=True,
                                   max_gamma = max_gamma,
                                   minimum_peak_height=minimum_peak_height,
                                   minimum_peak_distance=minimum_peak_distance,
                                   g_gamma=0.2,remove_hom=True,hom_max =hom_max,
                                   plot_fit=False,
                                   fit_peaks=True, save_fig=True)
# print 3.e5/x0s

nu_fsrs=np.array([])

# x0s=x0s[:-1]

for l in np.arange(len(x0s)-1):
    fund_1 = x0s[l]
    fund_2 = x0s[l+1]
    nu_fsr = np.abs(fund_2 - fund_1)*1.e12
    length = c/(2*nu_fsr)
    print 'FSR is ', nu_fsr*1.e-12, 'THz; ','length is ',length *1.e6 , 'um'
    if length>(cavlength_minrange*1.e-6) and length<(cavlength_maxrange*1.e-6):
        nu_fsrs = np.append(nu_fsrs,nu_fsr)
    else:
        print 'disregarded'
        pass

# print nu_fsrs
avg_nu_fsr = np.average(nu_fsrs)
u_avg_nu_fsr = scipy.stats.sem(nu_fsrs)
# print avg_nu_fsr
# avg_nu_fsrs[i,0] = avg_nu_fsr 
# avg_nu_fsrs[i,1] = u_avg_nu_fsr

length = c/(2*avg_nu_fsr)
u_length = c/(2*avg_nu_fsr**2)*u_avg_nu_fsr

print 'the average length is ',length *1.e6,'+-',u_length*1.e6 , 'um' 


In [ ]:
reload(spectrometer_analysis_2Dsweep)
reload(sa)

# data_dir=r'D:\measuring\data\20170914\180800_spectrometer_linewidth_maybe_exposure_30s'
data_dir=r'D:\measuring\data\20170921\185800_cav_spectrum'



sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,load_analysis_from_file=False)#,V_min=ana_pars['V_min'],V_max=ana_pars['V_max'],laser_wavelength=ana_pars['laser_wavelength'])
sa_2D.ana_pars['min_frq']=470# 471.8
sa_2D.ana_pars['max_frq']=471# 472.7

frequencies,filenumbers,intensities=sa_2D.get_data()

# import pylab as plb
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

def gaus(x,a,x0,sigma,b):
    return a*exp(-(x-x0)**2/(2*sigma**2))+b

                

x = frequencies
y = intensities.flatten()
popt,pcov = curve_fit(gaus,x,y,p0=[ np.max(y), x[ np.argmax( y ) ], 0.030, y[0] ] )    
sigma = popt[2]        
yFit = gaus(x,*popt)    


plt.plot( frequencies, intensities, '.' )
plt.plot( frequencies, yFit, '-' )
plt.title( '{} gauss sigma {:.1f} GHz'.format( data_dir, s * 1e3 ))
plt.grid()
plt.show()



### Extract effective radius of curavature

In [ ]:
reload(spectrometer_analysis_2Dsweep)
reload(sa)

# use same as above where cav length determined. this cell uses 'length' as determined there
# data_dir=r'D:\measuring\data\20170718\135100_N16_spectrometer'

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,load_analysis_from_file=False)#,V_min=ana_pars['V_min'],V_max=ana_pars['V_max'],laser_wavelength=ana_pars['laser_wavelength'])
sa_2D.ana_pars['min_frq']=440
sa_2D.ana_pars['max_frq']=550

frequencies,filenumbers,intensities=sa_2D.get_data()

plt.plot(frequencies,intensities)
plt.yscale('log')

nr_skip = 0
nr_modes = 6
minimum_peak_height = 00
minimum_peak_distance = 20
max_gamma = 20

x0s,u_x0s,n_trans, u_n_trans =sa_2D.peaks_from_1D_data(intensity=sa_2D.intensities[:,0],plot_peak_locations=True,
                                   max_gamma = max_gamma,
                                   minimum_peak_height=minimum_peak_height,
                                   minimum_peak_distance=minimum_peak_distance,
                                   g_gamma=0.2,remove_hom=False,hom_max =10,
                                   plot_fit=False,
                                   fit_peaks=True, save_fig=True)

x0s = np.sort(x0s)
x0s = x0s[nr_skip:]*1.e12

rocs=[]
print 'cavity length:',length*1e6,'um'

for l in np.arange(len(x0s)):
    if l%nr_modes==0:
        fund = x0s[l]
        print 'fundamental',fund
    else:
        nr_hom = l%nr_modes
        
        dnu_01 = x0s[l]-fund
#         print nr_hom
#         print x0s[l]
#         print dnu_01
#         print (1-(np.cos( (4*math.pi*dnu_01*length)/((2*nr_hom)*c) )**2))
#         roc = length/(1-(np.cos( (4*math.pi*dnu_01*length)/(4*c) )**2))
        roc = length/(1-(np.cos( (4*math.pi*dnu_01*length)/((2*nr_hom)*c) )**2))

        print 'ROC',roc*1.e6
        rocs.append(roc)     
    
avg_roc = np.average(rocs)
u_avg_roc = scipy.stats.sem(rocs)
np.savetxt(data_dir+'/ROCs.txt',np.array(rocs))
np.savetxt(data_dir+'/avg_ROC.txt',np.array([avg_roc,u_avg_roc]))

print 'the average roc is ',round(avg_roc *1.e6,2),'+-',round(u_avg_roc*1.e6,2) , 'um' 
       



In [ ]:
#plotting ROCs vs cavity length
ROCs = [39.16,38.46,39.2,39.07,37.8,39.52,43.86]
u_ROCs = [0.54,0.4,0.49,0.58,0.58,0.56,0.5]
Ls = [11.74,8.85,6.00,5.04,15.88,12.35,14.3]

# average over all points from the same day
print np.average(ROCs[:-1]),'um'
print scipy.stats.sem(ROCs[:-1]),'um'

plt.errorbar(Ls,ROCs,yerr=u_ROCs,fmt='o')
plt.xlabel('cavity length (um)')
plt.ylabel('measured ROC (um)')

### nr_datafiles = 1
minimum_peak_heights = [6000,12000]
avg_nu_fsrs = np.zeros((nr_datafiles,2))
min_fsrs = np.ones(nr_datafiles)*4*1.e12
max_fsrs = np.ones(nr_datafiles)*20.e12
#max_fsrs[5] = 40.e12
#max_fsrs[6] = 40.e12
#max_fsrs[7] = 50.e12
#max_fsrs[8] = 50.e12
#min_fsrs[7] = 10.e12
#min_fsrs[8] = 10.e12

reload(sa_2D)
filename = '2016 september 07 18_01_47 OND_LT_POS2_L4 1185.csv'

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies,filenumbers,intensities = sa_2D.get_data(min_frq=432,max_frq=496)#load_data_from_folder(data_dir)


for i in np.arange(nr_datafiles):
#    data_folder = os.path.join(data_dir,'L'+str(i))
    datafile = os.path.join(data_dir, filename)

    #note that what are called wavelengths, are actually the frequencies in THz! 
    wavelengths,intensities = sa.load_data(datafile)

    minimum_peak_height = minimum_peak_heights[i]
    x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,0],
                                       data_dir=datafile,save_fig=False,
                                       plot_fit =False, plot_peak_locations=True,minimum_peak_distance=10,
                                       minimum_peak_height=minimum_peak_height, g_gamma=0.1)
    print x0s

    # reinitialise every time
    nu_fsrs=np.array([])

    if i == 4:
        x0s=x0s[:-2]
    
    for l in np.arange(len(x0s)-1):
        fund_1 = x0s[l]
        fund_2 = x0s[l+1]
        nu_fsr = np.abs(fund_2 - fund_1)*1.e12
        if ((nu_fsr>min_fsrs[i]) and (nu_fsr<max_fsrs[i])):
            length = c/(2*nu_fsr)
            print nu_fsr*1.e-12, 'THz'
            print length *1.e6 , 'um'
            nu_fsrs = np.append(nu_fsrs,nu_fsr)
            
    print nu_fsrs
    avg_nu_fsr = np.average(nu_fsrs)
    u_avg_nu_fsr = np.std(nu_fsrs)
    print avg_nu_fsr
    avg_nu_fsrs[i,0] = avg_nu_fsr 
    avg_nu_fsrs[i,1] = u_avg_nu_fsr
            
        
#     fund_1 = x0s[0]
#     fund_2 = x0s[3]
#     dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
#     if dnu_fsr>2*1.e12:
#         length = c/(2*dnu_fsr)
#         print dnu_fsr*1.e-12, 'THz'
#         print length *1.e6 , 'um'



# wavelengths,filenumbers,intensities = sa.load_data(data_file)


f = open(os.path.join(data_dir,'fsr_data.txt'), 'w')
np.savetxt(f,avg_nu_fsrs)
f.close()

In [ ]:
# nus = [430.99,449.51,468.4,487.4]
# nus = [431.07,449.61,468.54,487.53]
# nus = [431.3,486.64]
# nus = [438.38, 495.11]
# nus = [441.52,471.33]
# nus=[434.48,470.77]
# nus = [451.34,484.24]
nus = [438.17,467.62,497.43]
nu_fsrs = np.array([])

for i,nu in enumerate(nus):
    if i < len(nus)-1:
        print i
        nu_fsr = nus[i+1] - nus[i]
        nu_fsrs =np.append(nu_fsrs,nu_fsr)
print nu_fsrs
avg_nu_fsrs = np.average(nu_fsrs*1.e12)
u_avg_nu_fsr = np.std(nu_fsrs*1.e12)
length = c/(2*avg_nu_fsrs)
print 'cavity length is ',length


In [ ]:
2.0*2.4*4e-6/36

In [ ]:

nu_fsrs = np.loadtxt(os.path.join(data_dir,'fsr_data.txt'))

linewidths = np.loadtxt(os.path.join(data_dir,'linewidth_data.txt'))



In [ ]:
fig,ax = plt.subplots()

ax.errorbar(nu_fsrs[:,0]*1.e-14,linewidths[:,0],xerr = nu_fsrs[:,1]*1.e-14,yerr=linewidths[:,1])
ax.set_xlabel('nu FSR (THz)')
ax.set_ylabel('linewidth (GHz)')

plt.show()

fig1,ax1 = plt.subplots()

ax1.errorbar(c/(2*nu_fsrs[:,0])*1.e6,np.divide(nu_fsrs[:,0],linewidths[:,0]*1.e9),yerr=linewidths[:,1])
ax1.set_xlabel('cavity length (um)')
ax1.set_ylabel('Finesse')


In [ ]:
reload(sa_2D)
minimum_peak_height = 6500
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,0],data_dir=data_dir,save_fig=False,plot_fit =False, plot_peak_locations=True,minimum_peak_height=minimum_peak_height, g_gamma=1)




In [ ]:
#Finding cavity length from peaks

for i in np.arange(len(x0s)-1):
    fund_1 = x0s[i]
    fund_2 = x0s[i+1]
    dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
    if dnu_fsr>3*1.e12:
        length = c/(2*dnu_fsr)
        print dnu_fsr*1.e-12, 'THz'
        print length *1.e6 , 'um'


fund_1 = x0s[0]
fund_2 = x0s[1]
dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
if dnu_fsr>2*1.e12:
    length = c/(2*dnu_fsr)
    print dnu_fsr*1.e-12, 'THz'
    print length *1.e6 , 'um'


In [ ]:
###analysing SPE files

In [ ]:
%matplotlib inline
import os
import numpy as np
import sys
import matplotlib
import matplotlib.pyplot as plt
c=2.998e8 #speed of light
import time
filepath='D:/measuring/'
import scipy.signal
os.chdir(filepath)

sys.path.append('H:/My Documents/measuring/')

data_dir='K:\ns\qt\Diamond\Samples\Cavity samples/2016-06 Sophie/2016 september 14 16_36_23zpl nv area 5 sophie spectrometer'#:\measuring\data/20160503\ON diamond\pos 7/raw data spectrometer\L9'#'D:\measuring\data/20160707\CL2_newmirror_processed'#"/Users/suzannevandam/Documents/PhD/localdata/20160430/ON_diamond" 
print data_dir

%matplotlib inline

import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as spectrometer_analysis_2Dsweep
#sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies,filenumbers,intensities = sa_2D.get_data(min_frq=468,max_frq=473)#load_data_from_folder(data_dir)
ax,fig = plt.subplots(figsize=(10,4))
plt.plot(frequencies,intensities[:,1])
ax.set_xlabel('Frequency (THz)')

In [ ]:
n = 2.4
lam = 637e-9
m = 30.0

L = m*lam/(2*n)
print L

In [ ]:
freq_in_GHz = (frequencies-470.4)*1e3
print freq_in_GHz[0]
print np.searchsorted(freq_in_GHz,[-200,300])
A = [3,2,1,0,-1,-2,-3]
B = A[::-1]
print A, B

idx = np.zeros(2)

print B[np.searchsorted(B,[-1.5],side='left')]



In [ ]:
import math
dx=12
dy=50
d = math.sqrt(dx**2+dy**2)
print d

dmap = 4.5 #cm

dconvert = dmap/d #in cm/micron
print dconvert
# daim = 13 #cm





In [ ]:
#calculate angle of diamond w.r.t objective coordinate frame
import numpy as np
angle = np.arctan(30/9)
print angle*180/math.pi



In [ ]:
xHmap = 11.4
yHmap = 6.7



Dx = yHmap*math.sin(angle)-xHmap*math.cos(angle)
Dy = yHmap*math.cos(angle)+xHmap*math.sin(angle)

Dyobj = Dy / dconvert
Dxobj = Dx / dconvert

print Dxobj,Dyobj 

In [ ]:
print 155+Dxobj
print -130+Dyobj

In [ ]:
math.sqrt((24.4-18.6)**2+(25.4-16)**2)